In [1]:
import json
import pandas as pd
import cv2
import numpy as np
from enum import Enum, auto
import sys

sys.path.append("../Skin_segmentation")
sys.path.append('../Models')

from Skindetector import *
from rPPGNet import *

with open('../../Data/json_structure') as json_file:
    data = json.load(json_file)


root_dir = "/work3/s174159/data/"
video_file = root_dir + "00/01/c920-1.avi"
bb_file = root_dir + "bbox/00/01/c920-1.face"

bb_data =  pd.read_csv(bb_file, sep=" ", header=None, names=["frame", "x", "y", "w", "h"]).drop("frame", axis=1)

In [ ]:
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

In [2]:
import cv2
from IPython.display import display, Image

# Define the VideoCapture object and VideoWriter
output_file = 'output_video.mp4'
video = cv2.VideoCapture(video_file)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for the output video
out = cv2.VideoWriter(output_file, fourcc, 30, (128, 128) ) # 30 is the frames per second, (128, 128) is the frame size

display_handle = display(None, display_id=True)
i = 0

try:
    while True:
        ret, frame = video.read()
        if not ret:  # Check if there are no more frames
            print("Finished video...")
            break  # Exit the loop if there are no more frames
        
        if i == len(data):
            i = 0
        x, y, w, h = bb_data.iloc[i].values.astype(int)
        i += 1
        frame = frame[y:y + h, x:x + w]
        frame = cv2.resize(frame, (128, 128), interpolation=cv2.INTER_AREA)
        detector = SkinDetector(frame)
        detector.find_skin()
        image, mask, skin = detector.get_resulting_images()
        
        # Save the processed frame to the output video
        out.write(skin)
        
        _, frame = cv2.imencode('.jpeg', skin)
        display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    video.release()
    out.release()  # Release the VideoWriter
    display_handle.update(None)


None

Finished video...
